In [ ]:
import json
with open('/home/sajadi/backup/tmp/wsd/detailedreslog.txt') as f:
    for line in f:
        jsline = line.split('\t')[1]
        js = json.loads(jsline)
        if ("wiki-mentions.5000.json" in js["dsname"]) and (js["ws"]==10):
            if js["method"]=='wlm':
                wlm = js["tp"]
            if js["method"]=='rvspagerank':
                rvs = js["tp"]
            if js["method"]=='ams':
                ams = js["tp"]              

In [ ]:
def get_prec(tp_list):
    overall_tp = 0
    overall_count=0
    macro_prec = 0;
    for tp, count in tp_list:
        overall_tp += tp
        overall_count += count
        macro_prec += float(tp)/count
        
    macro_prec = macro_prec/len(tp_list)
    micro_prec = float(overall_tp)/overall_count
    
    return micro_prec, macro_prec

In [ ]:
b=0
e=len(wlm)
e=5000

In [ ]:
print round(get_prec(wlm[b:e]))

In [ ]:
print get_prec(ams[b:e])

In [ ]:
print get_prec(rvs[b:e])

In [ ]:
round(.5)

# Some statistics for the paper

In [ ]:
l=0
i=0
with open('/home/sajadi/backup/tmpbackup/log.out.iterations.txt') as f:
    for line in f:
        r= re.match(".*# of iterations: ([\d]+),.*",line)
        if r is None:
            continue
        i += int(r.group(1))
        l +=1
print i,l, float(i)/l  


In [ ]:
(6.5+7.4)/2

# Processing wsd results

In [ ]:
import sys
table_no=1
sys.path.insert(0,'..')
from wikisim.utils import *

In [ ]:
import pandas as pd

#don't remember why had header
#df = pd.read_csv('/home/sajadi/backup/tmp/results/wsd/reslog.csv', sep='\t', header=0)

df = pd.read_csv('/home/sajadi/backup/tmp/results/wsd/reslog.csv', sep='\t', header = None)
df.columns=['start','method', 'op_method','dir','max_t', 'ws', 'dsname', 'micro','macro', 'time']
df.head()

In [ ]:
df1= df.drop('start', axis=1)
df1= df1.drop('ws', axis=1)
# df.drop(8, axis=1, inplace=True)

In [ ]:
df1.head()

In [ ]:
df2 = df1.copy()
df2.loc[:,'dsname'] = df2.loc[:,'dsname'].replace('/users/grad/sajadi/backup/datasets/ner/kore.json','KORE')
df2.loc[:,'dsname'] = df2.loc[:,'dsname'].replace('/users/grad/sajadi/backup/datasets/ner/aida.json','AIDA-CoNLL')
df2.loc[:,'dsname'] = df2.loc[:,'dsname'].replace('/users/grad/sajadi/backup/datasets/ner/wiki-mentions.5000.json','Wiki')
df2.loc[:,'dsname'] = df2.loc[:,'dsname'].replace('/users/grad/sajadi/backup/datasets/ner/msnbc.json','msnbc')
df2.loc[:,'dsname'] = df2.loc[:,'dsname'].replace('/users/grad/sajadi/backup/datasets/ner/aquaint.json','aquaint')

df2.loc[:,'method'] = df2.loc[:,'method'].replace('ams','overlap')
df2.loc[:,'method'] = df2.loc[:,'method'].replace('rvspagerank','rvsPageRank')
df2.loc[:,'method'] = df2.loc[:,'method'].replace('word2vec.300','word2vec')
df2.loc[:,'method'] = df2.loc[:,'method'].replace('word2vec.ehsan','word2vec')

df2.loc[:,'op_method'] = df2.loc[:,'op_method'].replace('ilp2','key-ilp')
df2.loc[:,'op_method'] = df2.loc[:,'op_method'].replace('context4_4','context')

df2.head()

In [ ]:
def methodselect(r):
    
    if table_no==0: #just for test
        #return ((r.loc['method'] == 'rvsPageRank') and (r.loc['dir']=='both') and (r.loc['op_method'] in ['context', 'tagme']))
        return (r.loc['op_method'] in ['tagme'])
    
    # Comparison methods among ilp
    if table_no==1: 
        return ((r.loc['method'] in ['wlm']) and (r.loc['dir']=='in') or \
                (r.loc['method'] in ['word2vec']) or \
                (r.loc['method'] in ['overlap', 'rvsPageRank']) and (r.loc['dir']=='both')) and \
                (r.loc['op_method'] == 'ilp')
        
    #How does Key-entity idea work    
    if table_no==2:
        return (r.loc['method'] == 'rvsPageRank' and( r.loc['dir']=='both') and \
                (r.loc['op_method'] in ['ilp','keyq'])) 
        #(r.loc['method'] == 'wlm' and r.loc['dir']=='in' and r.loc['op_method'] in  ['tagme'])
    #3: Time     
    if table_no==3:
        return ((r.loc['method'] == 'rvsPageRank') and (r.loc['dir']=='both') and \
                (r.loc['op_method'] in ['ilp','keydisamb', 'keyq'])) #or \
                #(r.loc['method'] in ['word2vec'] and r.loc['op_method'] in  ['context'])
    # relatedness vs VSM
    if table_no==4:
        return (r.loc['method'] == 'rvsPageRank' and( r.loc['dir']=='both') and \
                (r.loc['op_method'] in ['ilp' ,'entitycontext','keydisamb', 'keyq']))
    # VSM: word2vec vs rvspagerank
    if table_no==5:
        return (r.loc['method'] == 'rvsPageRank' and( r.loc['dir']=='both') and \
                (r.loc['op_method'] in ['entitycontext','keydisamb'])) or \
                ('word2vec' in r.loc['method'] and r.loc['op_method'] in ['entitycontext','keydisamb'])
    
df3 = df2[df2.apply(lambda r: ('wiki-mentions.json' not in r.loc['dsname']) and (r.loc['max_t'] in [5,10,15]), axis=1)]
df3 = df3[df3.apply(methodselect, axis=1)]
df3

In [ ]:
df4=df3.drop('dir', axis=1)
if table_no in [1]:
    df4=df4.drop('op_method', axis=1)
if table_no==2:
    df4[df4['method']=='word2vec']=df4[df4['method']=='word2vec'].replace('keydisamb', 'key-vsm-word2vec')
    df4['op_method']=df4['op_method'].replace('keyq', 'key')
    df4['op_method']=df4['op_method'].replace('keydisamb', 'key-vsm')
    df4=df4.drop('method', axis=1)
if table_no==3:
    df4=df4.drop(['method', 'micro', 'macro'], axis=1)
if table_no==4:
    df4=df4.drop('method', axis=1)
    
    
df4

In [ ]:
cols = df4.columns.tolist()
cols

In [ ]:
cols = df4.columns.tolist()
if table_no in [1,2,3]:
    cols[0],cols[1] = cols[1],cols[0]
if table_no in [4]:
    #cols[0],cols[1], cols[2] = cols[2],cols[0],cols[1]
    cols[0],cols[1] = cols[1],cols[0]
if table_no in [5]:
    cols[0],cols[1], cols[2] = cols[2],cols[1],cols[0]
    
df5 = df4[cols]
df5.head()

In [ ]:
if table_no==3:
    df5.loc[:,'time'] = df5.loc[:,'time'].apply(lambda x: str2delta(x).total_seconds())
df5    

In [ ]:
#df5 = df5.round(2)

In [ ]:
if table_no==1:
    grouped = df5.groupby(['max_t','method'])
if table_no in [0,2,3]:
    grouped = df5.groupby(['max_t','op_method'])
if table_no in [4]:
    grouped = df5.groupby(['max_t','op_method'])
if table_no in [5]:
    grouped =  df5.groupby(['max_t','op_method'])
    grouped =  df5.groupby(['max_t','op_method', 'method'])
    #grouped = df5.groupby(['max_t','method','dsname'])
#grouped.get_group((5,'ams'))

In [ ]:
for g in grouped.groups:
    print grouped.get_group(g)
    print '----------------------'
    

In [ ]:
gdf = grouped.get_group(g)

tkore = gdf[gdf['dsname'] == "KORE"]
taida = gdf[gdf['dsname'] == "AIDA-CoNLL"]
twiki = gdf[gdf['dsname'] == "Wbiki"]
tmsnbc = gdf[gdf['dsname'] == "msnbc"]
taquaint = gdf[gdf['dsname'] == "aquaint"]
print gdf
print "--------"
print tkore
# tdf = pd.DataFrame([[gdf['max_t'].iloc[0], gdf['method'].iloc[0]] 
#                     + list(tmsnbc.iloc[0,micmac:micmac+l]) + list(taquaint.iloc[0,micmac:micmac+l])+ list(tkore.iloc[0,micmac:micmac+l])+ list(taida.iloc[0,micmac:micmac+l]) + list(twiki.iloc[0,micmac:micmac+l]) ])


In [ ]:
#3 = mic, 4=mac?
micmac=3
l=2
if table_no==5:
    micmac += 1
rdf = pd.DataFrame()
for g in grouped.groups:
    gdf = grouped.get_group(g)
    tkore = gdf[gdf['dsname'] == "KORE"]
    taida = gdf[gdf['dsname'] == "AIDA-CoNLL"]
    twiki = gdf[gdf['dsname'] == "Wiki"]
    tmsnbc = gdf[gdf['dsname'] == "msnbc"]
    taquaint = gdf[gdf['dsname'] == "aquaint"]
    if table_no==1:
        tdf = pd.DataFrame([[gdf['max_t'].iloc[0], gdf['method'].iloc[0]] 
                        + list(tmsnbc.iloc[0,micmac:micmac+l]) + list(taquaint.iloc[0,micmac:micmac+l])+ list(tkore.iloc[0,micmac:micmac+l])+ list(taida.iloc[0,micmac:micmac+l]) + list(twiki.iloc[0,micmac:micmac+l]) ])
    if table_no in [0, 2, 4]:
        tdf = pd.DataFrame([[gdf['max_t'].iloc[0], gdf['op_method'].iloc[0]] 
                        + list(tmsnbc.iloc[0,micmac:micmac+l]) + list(taquaint.iloc[0,micmac:micmac+l]) + list(tkore.iloc[0,micmac:micmac+l])+ list(taida.iloc[0,micmac:micmac+l]) + list(twiki.iloc[0,micmac:micmac+l]) ])
    if table_no==3:
        tdf = pd.DataFrame([[gdf['max_t'].iloc[0], gdf['op_method'].iloc[0]] 
                        + list(tmsnbc.iloc[0,3:]) + list(taquaint.iloc[0,3:]) + list(tkore.iloc[0,3:])+ list(taida.iloc[0,3:]) + list(twiki.iloc[0,3:])  ])
    if table_no in [5]:
        tdf = pd.DataFrame([[gdf['max_t'].iloc[0], gdf['op_method'].iloc[0], gdf['method'].iloc[0]] 
                        + list(tmsnbc.iloc[0,micmac:micmac+l]) + list(taquaint.iloc[0,micmac:micmac+l]) + list(tkore.iloc[0,micmac:micmac+l])+ list(taida.iloc[0,micmac:micmac+l]) + list(twiki.iloc[0,micmac:micmac+l]) ])
    
    rdf = rdf.append(tdf)

In [ ]:
rdf

In [ ]:
#miindex = pd.MultiIndex.f(index, names=['cs', 'SR Method'])
midf = rdf.copy()
if table_no==1:
    midf[1]=pd.Categorical(midf[1],['overlap','wlm','word2vec','rvsPageRank'])
if table_no==2:
    midf[1]=pd.Categorical(midf[1],['ilp','key','key-vsm-word2vec', 'key-vsm'])
if table_no==4:
    midf[1]=pd.Categorical(midf[1],['ilp','entitycontext', 'keyq','keydisamb' ])
if table_no==5:
    midf[1]=pd.Categorical(midf[1],['entitycontext', 'keydisamb' ])
    midf[2]=pd.Categorical(midf[2],['word2vec','rvsPageRank'])
    
midf = midf.sort_values(by=[0,1,2])
midf

In [ ]:
midf.index = pd.MultiIndex.from_arrays([midf[0], midf[1]],names=['cs', 'Method'])
if table_no ==5:
    midf.index = pd.MultiIndex.from_arrays([midf[0], midf[1],midf[2]],names=['cs', 'Op Method', 'Method'])

midf

In [ ]:
midf.drop(0, axis=1, inplace=True)
midf.drop(1, axis=1, inplace=True)
if table_no==5:
    midf.drop(2, axis=1, inplace=True)
midf

In [ ]:
C=["MSNBC", "AQUAINT", "KORE","AIDA-CoNLL","Wiki"]
micolumns = pd.MultiIndex.from_arrays([[y  for x in zip(C,C) for y in x], ["MiAP","MAP"]*5],names=['cs', 'Method'])

midf.columns = micolumns
midf = midf.round(2)
midf

In [ ]:
print midf.to_latex()

In [ ]:
midf_plot = midf.copy()
midf_plot=midf_plot.drop('AQUAINT', axis=1)
midf_plot=midf_plot.drop('KORE', axis=1)
#midf_plot=midf_plot.drop('MSNBC', axis=1)

#midf = midf.drop['']

In [ ]:
x=pd.Series()
y=pd.Series()
z=pd.Series()
l=[]
for i in midf_plot.index.levels[0]:
        x=x.append(midf_plot.loc[(i,'ilp')])
        y=y.append(midf_plot.loc[(i,'keyq')])
        z=z.append(midf_plot.loc[(i,'keydisamb')])
        l += [c+'\n'+str(i) for c in midf_plot.columns]
x=x.values
y=y.values
z=z.values
        

In [ ]:
import numpy as np
x=np.log10(x)
y=np.log10(y)
z=np.log10(z)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(num=None, figsize=(7, 5), dpi=80, facecolor='w', edgecolor='k')

index = np.arange(len(x))
#bar_width = 0.45

opacity = 0.4
error_config = {'ecolor': '0.3'}

plt.plot(index, x, '-.o',  ms=8, lw=2,alpha=0.7, color='red', label='Coherence (ILP)')
plt.plot(index, y, '-v', ms=8, lw=2,alpha=0.7, color='green', label='Key Search')
plt.plot(index, z, ':*', ms=8, lw=2,alpha=0.7, color='blue', label='VSM Key Recog.')

plt.xlabel('Datasets and Candidate Numbers')
plt.ylabel('Time log(Seconds)')


# l1=["MSNBC", "AQUAINT", "KORE","AIDA-CoNLL","Wiki"]*3
# l2=['5']*5+['10']*5+['15']*5

l1=["MSNBC","AIDA-CoNLL","Wiki"]*3
l2=['5']*3+['10']*3+['15']*3


labels = [r+"\n"+s for r,s in zip(l1,l2)]
plt.xticks(np.arange(len(x)),labels)
plt.legend()

plt.tight_layout()
plt.legend(loc=2)
plt.savefig('timebarlog.eps')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(num=None, figsize=(9, 5), dpi=80, facecolor='w', edgecolor='k')

index = np.arange(len(x))
bar_width = 0.32

opacity = 0.4
error_config = {'ecolor': '0.3'}

# plt.plot(index, x, '-.o',  ms=10, lw=3,alpha=0.7, color='red', label='Coherence (ILP)')
# plt.plot(index, y, '-v', ms=10, lw=3,alpha=0.7, color='green', label='Key Search')
# plt.plot(index, z, ':*', ms=10, lw=3,alpha=0.7, color='blue', label='VSM Key Recog.')
rects1 = plt.bar(index, x, bar_width,
                 alpha=opacity,
                 color='b',
                
                 error_kw=error_config)

rects2 = plt.bar(index + bar_width, y, bar_width,
                 alpha=opacity,
                 color='r',
                 error_kw=error_config)
rects2 = plt.bar(index - bar_width, z, bar_width,
                 alpha=opacity,
                 color='g')

plt.xlabel('Datasets and Candidate Numbers')
plt.ylabel('Time (Seconds)')

# l1=["DS1","DS2","DS3","DS4","DS5"]*3
# l2=['5']*2+['10']*2+['15']*5

l1=["AIDA","WIKI"]*3
l2=['5']*2+['10']*2+['15']*5


labels = [r+"\n"+s for r,s in zip(l1,l2)]
plt.xticks(np.arange(len(x)),labels)
plt.legend()

plt.tight_layout()
plt.legend(loc=2)
plt.savefig('timebar.eps')
plt.show()


# Walk Through Example

In [1]:
# %load_ext autoreload
# %autoreload

# %aimport wsd
import sys
sys.path.insert(0,'..')
from wsd.wsd import *
import time
ws=5
word2vec_path = os.path.join(home, 'backup/wikipedia/WikipediaClean5Negative300Skip10.Ehsan/WikipediaClean5Negative300Skip10')
#gensim_loadmodel(word2vec_path)
#print "loaded"
#sys.stdout.flush()

S=["David", "started", "dating", "Victoria" , "after", "she", "attended", "a", "Manchester United", "match"]
M=[[0, "David_Beckham"], [3, "Victoria_Beckham"], [8, "Manchester United F.C."]]

direction=DIR_BOTH
method='rvspagerank'

start = time.time()
C = generate_candidates(S, M, max_t=4, enforce=True)
print "Candidates: ", C, "\n"
candslist=C

Candidates:  [[(8551L, 0.8361233480176211), (36684L, 0.07048458149779736), (9072L, 0.06607929515418502), (8618L, 0.027312775330396475)], [(4689460L, 0.8320106749069457), (32388L, 0.08603132242432755), (1147963L, 0.08097478755530585), (45979L, 0.0009832151134208864)], [(19961L, 0.9951503394762367), (9768801L, 0.004156851877511431), (14510069L, 0.00041568518775114314), (45558839L, 0.0002771234585007621)]] 



In [2]:
idmap={}
index=[]
for i in range(len(C)):
    for j in range(len(C[0])):
        idmap[C[i][j][0]]= '$C_{%s%s}$'%(i,j)
        index.append('$C_{%s%s}$'%(i,j))
        
print idmap
print index

{9768801L: '$C_{21}$', 45979L: '$C_{13}$', 32388L: '$C_{11}$', 8551L: '$C_{00}$', 8618L: '$C_{03}$', 36684L: '$C_{01}$', 9072L: '$C_{02}$', 4689460L: '$C_{10}$', 14510069L: '$C_{22}$', 45558839L: '$C_{23}$', 19961L: '$C_{20}$', 1147963L: '$C_{12}$'}
['$C_{00}$', '$C_{01}$', '$C_{02}$', '$C_{03}$', '$C_{10}$', '$C_{11}$', '$C_{12}$', '$C_{13}$', '$C_{20}$', '$C_{21}$', '$C_{22}$', '$C_{23}$']


In [3]:
candf=pd.DataFrame()
for i in range(len(C[0])):
    pf=pd.DataFrame([["%s (%s)"%(idmap[C[j][i][0]], id2title(C[j][i][0])) for j in range(len(C))]])
    candf=candf.append(pf)
candf.columns=[S[i] for i,_ in M]
candf.index=["$C_{%dj}$"%i for i in range(len(C[0]))]
candf

,David,Victoria,Manchester United
$C_{0j}$,$C_{00}$ (David),$C_{10}$ (Victoria_(Australia)),$C_{20}$ (Manchester_United_F.C.)
$C_{1j}$,$C_{01}$ (David_(Michelangelo)),"$C_{11}$ (Victoria,_British_Columbia)",$C_{21}$ (Manchester_United_F.C._Reserves_and_...
$C_{2j}$,$C_{02}$ (Jacques-Louis_David),$C_{12}$ (Victoria_cricket_team),$C_{22}$ (Manchester_United_(video_game_series))
$C_{3j}$,$C_{03}$ (David_Beckham),$C_{13}$ (Victoria_Beckham),$C_{23}$ (Manchester_Eagles)


In [4]:
print candf.iloc[1,2]

$C_{21}$ (Manchester_United_F.C._Reserves_and_Academy)


In [5]:

print candf.to_latex()

\begin{tabular}{llll}
\toprule
{} &                            David &                               Victoria &                                  Manchester United \\
\midrule
\$C\_\{0j\}\$ &                 \$C\_\{00\}\$ (David) &        \$C\_\{10\}\$ (Victoria\_(Australia)) &                  \$C\_\{20\}\$ (Manchester\_United\_F.C.) \\
\$C\_\{1j\}\$ &  \$C\_\{01\}\$ (David\_(Michelangelo)) &  \$C\_\{11\}\$ (Victoria,\_British\_Columbia) &  \$C\_\{21\}\$ (Manchester\_United\_F.C.\_Reserves\_and\_... \\
\$C\_\{2j\}\$ &   \$C\_\{02\}\$ (Jacques-Louis\_David) &       \$C\_\{12\}\$ (Victoria\_cricket\_team) &   \$C\_\{22\}\$ (Manchester\_United\_(video\_game\_series)) \\
\$C\_\{3j\}\$ &         \$C\_\{03\}\$ (David\_Beckham) &            \$C\_\{13\}\$ (Victoria\_Beckham) &                       \$C\_\{23\}\$ (Manchester\_Eagles) \\
\bottomrule
\end{tabular}



In [6]:
simmatrix = get_sim_matrix(C, 'rvspagerank', DIR_BOTH)
simindex = [idmap[c] for c in simmatrix.columns]
simmatrix.index=simindex
simmatrix.columns=simindex
simmatrix

,$C_{21}$,$C_{13}$,$C_{11}$,$C_{00}$,$C_{03}$,$C_{01}$,$C_{02}$,$C_{10}$,$C_{22}$,$C_{23}$,$C_{20}$,$C_{12}$
$C_{21}$,NaN,0.0013397,0.000447124,0,0.0840262,0.000106765,0.000265367,0.000512475,NaN,NaN,NaN,4.5928e-05
$C_{13}$,0.0013397,NaN,NaN,7.22615e-05,0.0850027,6.71967e-05,0.000116272,NaN,0,0,0.00192553,NaN
$C_{11}$,0.000447124,NaN,NaN,0.000113711,0.000916761,6.65689e-05,9.85359e-05,NaN,0.000118357,0.00130006,0.000628357,NaN
$C_{00}$,0,7.22615e-05,0.000113711,NaN,NaN,NaN,NaN,4.76975e-05,0,0,0.000149686,0
$C_{03}$,0.0840262,0.0850027,0.000916761,NaN,NaN,NaN,NaN,0.00017354,0.000652401,0.00123396,0.141464,2.13361e-05
$C_{01}$,0.000106765,6.71967e-05,6.65689e-05,NaN,NaN,NaN,NaN,0.000102302,0,0,0.000144055,0
$C_{02}$,0.000265367,0.000116272,9.85359e-05,NaN,NaN,NaN,NaN,7.15048e-05,0,0,0.00039319,0
$C_{10}$,0.000512475,NaN,NaN,4.76975e-05,0.00017354,0.000102302,7.15048e-05,NaN,0.000219056,0,0.000405134,NaN
$C_{22}$,NaN,0,0.000118357,0,0.000652401,0,0,0.000219056,NaN,NaN,NaN,0.000119457
$C_{23}$,NaN,0,0.00130006,0,0.00123396,0,0,0,NaN,NaN,NaN,0


In [7]:
df=simmatrix[index].loc[index]
df=df.apply(pd.to_numeric)
df=df.round(4)
df

,$C_{00}$,$C_{01}$,$C_{02}$,$C_{03}$,$C_{10}$,$C_{11}$,$C_{12}$,$C_{13}$,$C_{20}$,$C_{21}$,$C_{22}$,$C_{23}$
$C_{00}$,NaN,NaN,NaN,NaN,0.0000,0.0001,0.0000,0.0001,0.0001,0.0000,0.0000,0.0000
$C_{01}$,NaN,NaN,NaN,NaN,0.0001,0.0001,0.0000,0.0001,0.0001,0.0001,0.0000,0.0000
$C_{02}$,NaN,NaN,NaN,NaN,0.0001,0.0001,0.0000,0.0001,0.0004,0.0003,0.0000,0.0000
$C_{03}$,NaN,NaN,NaN,NaN,0.0002,0.0009,0.0000,0.0850,0.1415,0.0840,0.0007,0.0012
$C_{10}$,0.0000,0.0001,0.0001,0.0002,NaN,NaN,NaN,NaN,0.0004,0.0005,0.0002,0.0000
$C_{11}$,0.0001,0.0001,0.0001,0.0009,NaN,NaN,NaN,NaN,0.0006,0.0004,0.0001,0.0013
$C_{12}$,0.0000,0.0000,0.0000,0.0000,NaN,NaN,NaN,NaN,0.0002,0.0000,0.0001,0.0000
$C_{13}$,0.0001,0.0001,0.0001,0.0850,NaN,NaN,NaN,NaN,0.0019,0.0013,0.0000,0.0000
$C_{20}$,0.0001,0.0001,0.0004,0.1415,0.0004,0.0006,0.0002,0.0019,NaN,NaN,NaN,NaN
$C_{21}$,0.0000,0.0001,0.0003,0.0840,0.0005,0.0004,0.0000,0.0013,NaN,NaN,NaN,NaN


In [8]:
df=df.apply(lambda x: x*10000)

In [9]:
print df.to_latex()

\begin{tabular}{lrrrrrrrrrrrr}
\toprule
{} &  \$C\_\{00\}\$ &  \$C\_\{01\}\$ &  \$C\_\{02\}\$ &  \$C\_\{03\}\$ &  \$C\_\{10\}\$ &  \$C\_\{11\}\$ &  \$C\_\{12\}\$ &  \$C\_\{13\}\$ &  \$C\_\{20\}\$ &  \$C\_\{21\}\$ &  \$C\_\{22\}\$ &  \$C\_\{23\}\$ \\
\midrule
\$C\_\{00\}\$ &       NaN &       NaN &       NaN &       NaN &       0.0 &       1.0 &       0.0 &       1.0 &       1.0 &       0.0 &       0.0 &       0.0 \\
\$C\_\{01\}\$ &       NaN &       NaN &       NaN &       NaN &       1.0 &       1.0 &       0.0 &       1.0 &       1.0 &       1.0 &       0.0 &       0.0 \\
\$C\_\{02\}\$ &       NaN &       NaN &       NaN &       NaN &       1.0 &       1.0 &       0.0 &       1.0 &       4.0 &       3.0 &       0.0 &       0.0 \\
\$C\_\{03\}\$ &       NaN &       NaN &       NaN &       NaN &       2.0 &       9.0 &       0.0 &     850.0 &    1415.0 &     840.0 &       7.0 &      12.0 \\
\$C\_\{10\}\$ &       0.0 &       1.0 &       1.0 &       2.0 &       NaN &       NaN &       NaN

In [11]:
ids, titles = disambiguate_driver(C, ws, method='rvspagerank', op_method='ilp')
print titles
#printed score is ('Score:', 0.22839198794632598)

['David_Beckham', 'Victoria_Beckham', 'Manchester_United_F.C.']


In [12]:
res_all=[]
simmatrix = get_sim_matrix(C, 'rvspagerank', DIR_BOTH)
#print simmatrix
for i in range(len(candslist)):
    for j in range(len(candslist[i])):
        res_ij =  evalkey(candslist[i][j], i, candslist, simmatrix)
        res_all.append(res_ij)
print res_all        
res, score = max(res_all, key=lambda x: x[1])
print("Score:", score)
titles = ids2title(res)

# candslist_scores = coherence_scores_driver(C, ws, method='rvspagerank', direction=DIR_BOTH, op_method="simplecontext")
# print "Key Scores_method_2: ", candslist_scores, "\n"


# ids, titles = disambiguate_driver(S, M, C, ws, method='word2vec', op_method='simplecontext')
# tp = get_tp(M, ids) 

# candslist_scores = coherence_scores_driver(C, ws, method='rvspagerank', direction=DIR_BOTH, op_method="keydisamb")
# print "Key Scores_method_1: ", candslist_scores, "\n"

# candslist_scores = coherence_scores_driver(C, ws, method='rvspagerank', direction=DIR_BOTH, op_method="entitycontext")
# print "Key Scores_method_2: ", candslist_scores, "\n"


[([8551L, 32388L, 19961L], 0.00026339703514977053), ([36684L, 4689460L, 19961L], 0.00024635693152275095), ([9072L, 45979L, 19961L], 0.00050946228718629083), ([8618L, 45979L, 19961L], 0.22646645411840516), ([8618L, 4689460L, 9768801L], 0.00068601504434118166), ([8618L, 32388L, 45558839L], 0.0022168252343899297), ([8618L, 1147963L, 19961L], 0.00021065845738099664), ([8618L, 45979L, 19961L], 0.086928219037004761), ([8618L, 45979L, 19961L], 0.14338930273724204), ([8618L, 45979L, 9768801L], 0.085365895014948845), ([8618L, 4689460L, 14510069L], 0.00087145696474333345), ([8618L, 32388L, 45558839L], 0.0025340215140948752)]
('Score:', 0.22646645411840516)


In [13]:
res_all2=[]
for x in res_all:
    y=([idmap[cid] for cid in x[0]], x[1])
    res_all2.append(y)
res_all2            

[(['$C_{00}$', '$C_{11}$', '$C_{20}$'], 0.00026339703514977053),
 (['$C_{01}$', '$C_{10}$', '$C_{20}$'], 0.00024635693152275095),
 (['$C_{02}$', '$C_{13}$', '$C_{20}$'], 0.00050946228718629083),
 (['$C_{03}$', '$C_{13}$', '$C_{20}$'], 0.22646645411840516),
 (['$C_{03}$', '$C_{10}$', '$C_{21}$'], 0.00068601504434118166),
 (['$C_{03}$', '$C_{11}$', '$C_{23}$'], 0.0022168252343899297),
 (['$C_{03}$', '$C_{12}$', '$C_{20}$'], 0.00021065845738099664),
 (['$C_{03}$', '$C_{13}$', '$C_{20}$'], 0.086928219037004761),
 (['$C_{03}$', '$C_{13}$', '$C_{20}$'], 0.14338930273724204),
 (['$C_{03}$', '$C_{13}$', '$C_{21}$'], 0.085365895014948845),
 (['$C_{03}$', '$C_{10}$', '$C_{22}$'], 0.00087145696474333345),
 (['$C_{03}$', '$C_{11}$', '$C_{23}$'], 0.0025340215140948752)]

In [14]:
df2=pd.DataFrame()

for i in range(len(C[0])):
    pf=pd.DataFrame([["$C_{%dj}$"%i]+['Key']+["%s"%(idmap[C[j][i][0]],) for j in range(len(C))]])
    df2=df2.append(pf)
    #pf=pd.DataFrame([["%s"%(id2title(C[j][i][0]),) for j in range(len(C))]], index=["$C_{%dj}$"%i])
    #df2=df2.append(pf)

    pf=pd.DataFrame([["$C_{%dj}$"%i]+['Resolved Mentions']+["[%s]"%(",".join(res_all2[j*len(C[0])+i][0]),) for j in range(len(C))]])
    df2=df2.append(pf)
    pf=pd.DataFrame([["$C_{%dj}$"%i]+['Key-Coherence']+["%s"%round(res_all2[j*len(C[0])+i][1],4) for j in range(len(C))]])
    df2=df2.append(pf)
    
#    pf=pd.DataFrame([["%s (%s) (%s)"%(id2title(C[j][i][0]), res_all2[j*len(C[0])+i][0],round(res_all2[j*len(C[0])+i][1],4)) for j in range(len(C))]])

#    df2=df2.append(pf)
# df2.columns=[S[i] for i,_ in M]
# index_ = ["$C_{%dj}$"%i for i in range(len(C[0]))]
# #index = 
# df2.index=["$C_{%dj}$"%i for i in range(len(C[0]))]
#df2.index = pd.MultiIndex.from_arrays([df2[0], df2[1]])                                                                                    
#df2.drop(0, axis=1, inplace=True)
#df2.drop(1, axis=1, inplace=True)
df2

,0,1,2,3,4
0,$C_{0j}$,Key,$C_{00}$,$C_{10}$,$C_{20}$
0,$C_{0j}$,Resolved Mentions,"[$C_{00}$,$C_{11}$,$C_{20}$]","[$C_{03}$,$C_{10}$,$C_{21}$]","[$C_{03}$,$C_{13}$,$C_{20}$]"
0,$C_{0j}$,Key-Coherence,0.0003,0.0007,0.1434
0,$C_{1j}$,Key,$C_{01}$,$C_{11}$,$C_{21}$
0,$C_{1j}$,Resolved Mentions,"[$C_{01}$,$C_{10}$,$C_{20}$]","[$C_{03}$,$C_{11}$,$C_{23}$]","[$C_{03}$,$C_{13}$,$C_{21}$]"
0,$C_{1j}$,Key-Coherence,0.0002,0.0022,0.0854
0,$C_{2j}$,Key,$C_{02}$,$C_{12}$,$C_{22}$
0,$C_{2j}$,Resolved Mentions,"[$C_{02}$,$C_{13}$,$C_{20}$]","[$C_{03}$,$C_{12}$,$C_{20}$]","[$C_{03}$,$C_{10}$,$C_{22}$]"
0,$C_{2j}$,Key-Coherence,0.0005,0.0002,0.0009
0,$C_{3j}$,Key,$C_{03}$,$C_{13}$,$C_{23}$


In [15]:
print df2.to_latex()

\begin{tabular}{llllll}
\toprule
{} &         0 &                  1 &                             2 &                             3 &                             4 \\
\midrule
0 &  \$C\_\{0j\}\$ &                Key &                      \$C\_\{00\}\$ &                      \$C\_\{10\}\$ &                      \$C\_\{20\}\$ \\
0 &  \$C\_\{0j\}\$ &  Resolved Mentions &  [\$C\_\{00\}\$,\$C\_\{11\}\$,\$C\_\{20\}\$] &  [\$C\_\{03\}\$,\$C\_\{10\}\$,\$C\_\{21\}\$] &  [\$C\_\{03\}\$,\$C\_\{13\}\$,\$C\_\{20\}\$] \\
0 &  \$C\_\{0j\}\$ &      Key-Coherence &                        0.0003 &                        0.0007 &                        0.1434 \\
0 &  \$C\_\{1j\}\$ &                Key &                      \$C\_\{01\}\$ &                      \$C\_\{11\}\$ &                      \$C\_\{21\}\$ \\
0 &  \$C\_\{1j\}\$ &  Resolved Mentions &  [\$C\_\{01\}\$,\$C\_\{10\}\$,\$C\_\{20\}\$] &  [\$C\_\{03\}\$,\$C\_\{11\}\$,\$C\_\{23\}\$] &  [\$C\_\{03\}\$,\$C\_\{13\}\$,\$C\_\{21\}\$] \\
0 &  \$C

In [16]:
find_key_concept(candslist, direction, method, ver=4)

(array([[ 0.        ,  0.        ,  0.        , ...,  0.00033765,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.00035591,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ..., 
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]]),
 [(0, 4), (4, 8), (8, 12)],
 0,
 3,
 array([ 0.       ,  0.       ,  0.0002528, ...,  0.       ,  0.       ,  0.       ]))

In [17]:
cvec_arr, cveclist_bdrs, cands_score_list = entity_to_context_scores(candslist, direction, method);
cands_score_list

[[3.2522803941614775e-05,
  4.9116226914280858e-05,
  9.7660904438967577e-05,
  0.035346665168261238],
 [0.00031357799957065779,
  0.0010695516931025217,
  0.00010997074822205466,
  0.0074601870672248438],
 [0.038425563390561468,
  0.022959259012770961,
  0.0003132149913510629,
  0.0007079505454106183]]

In [18]:
K=[sorted(enumerate(k), key=lambda x: -x[1]) for k in cands_score_list]
K

[[(3, 0.035346665168261238),
  (2, 9.7660904438967577e-05),
  (1, 4.9116226914280858e-05),
  (0, 3.2522803941614775e-05)],
 [(3, 0.0074601870672248438),
  (1, 0.0010695516931025217),
  (0, 0.00031357799957065779),
  (2, 0.00010997074822205466)],
 [(0, 0.038425563390561468),
  (1, 0.022959259012770961),
  (3, 0.0007079505454106183),
  (2, 0.0003132149913510629)]]

In [19]:
SK=[[(idmap[C[i][j][0]], score) for j,score in (K[i])] for i in range(len(K))]
SK

[[('$C_{03}$', 0.035346665168261238),
  ('$C_{02}$', 9.7660904438967577e-05),
  ('$C_{01}$', 4.9116226914280858e-05),
  ('$C_{00}$', 3.2522803941614775e-05)],
 [('$C_{13}$', 0.0074601870672248438),
  ('$C_{11}$', 0.0010695516931025217),
  ('$C_{10}$', 0.00031357799957065779),
  ('$C_{12}$', 0.00010997074822205466)],
 [('$C_{20}$', 0.038425563390561468),
  ('$C_{21}$', 0.022959259012770961),
  ('$C_{23}$', 0.0007079505454106183),
  ('$C_{22}$', 0.0003132149913510629)]]

In [20]:
df3=pd.DataFrame()

for i in range(len(SK[0])):
    #pf=pd.DataFrame([["%s (%s)"%(id2title([C[j][i][0]]), res_all2[j*len(C[0])+i][1]) for j in range(len(C))]])
    pf=pd.DataFrame([["$C_{%dj}$"%i]+['Key']+["%s"%(SK[j][i][0],) for j in range(len(SK))]])
    df3=df3.append(pf)
    pf=pd.DataFrame([["$C_{%dj}$"%i]+['Context-Sim']+["%s"%(round(SK[j][i][1],4),) for j in range(len(SK))]])
    df3=df3.append(pf)
#df3.columns=[S2[i] for i,_ in M]
#df3.index=["$C_{%dj}$"%i for i in range(len(C[0]))]
df3

,0,1,2,3,4
0,$C_{0j}$,Key,$C_{03}$,$C_{13}$,$C_{20}$
0,$C_{0j}$,Context-Sim,0.0353,0.0075,0.0384
0,$C_{1j}$,Key,$C_{02}$,$C_{11}$,$C_{21}$
0,$C_{1j}$,Context-Sim,0.0001,0.0011,0.023
0,$C_{2j}$,Key,$C_{01}$,$C_{10}$,$C_{23}$
0,$C_{2j}$,Context-Sim,0.0,0.0003,0.0007
0,$C_{3j}$,Key,$C_{00}$,$C_{12}$,$C_{22}$
0,$C_{3j}$,Context-Sim,0.0,0.0001,0.0003


In [21]:
print df3.to_latex()

\begin{tabular}{llllll}
\toprule
{} &         0 &            1 &         2 &         3 &         4 \\
\midrule
0 &  \$C\_\{0j\}\$ &          Key &  \$C\_\{03\}\$ &  \$C\_\{13\}\$ &  \$C\_\{20\}\$ \\
0 &  \$C\_\{0j\}\$ &  Context-Sim &    0.0353 &    0.0075 &    0.0384 \\
0 &  \$C\_\{1j\}\$ &          Key &  \$C\_\{02\}\$ &  \$C\_\{11\}\$ &  \$C\_\{21\}\$ \\
0 &  \$C\_\{1j\}\$ &  Context-Sim &    0.0001 &    0.0011 &     0.023 \\
0 &  \$C\_\{2j\}\$ &          Key &  \$C\_\{01\}\$ &  \$C\_\{10\}\$ &  \$C\_\{23\}\$ \\
0 &  \$C\_\{2j\}\$ &  Context-Sim &       0.0 &    0.0003 &    0.0007 \\
0 &  \$C\_\{3j\}\$ &          Key &  \$C\_\{00\}\$ &  \$C\_\{12\}\$ &  \$C\_\{22\}\$ \\
0 &  \$C\_\{3j\}\$ &  Context-Sim &       0.0 &    0.0001 &    0.0003 \\
\bottomrule
\end{tabular}



In [22]:
R=[(k[0][0], (k[0][1]-k[1][1]) / k[1][1]) for k in SK]
R = [(r[0],round(r[1],4)) for r in R]
R

[('$C_{03}$', 360.9326), ('$C_{13}$', 5.9751), ('$C_{20}$', 0.6736)]